# Descripción de la notebook

- Levanta el archivo CSV generado en la notebook 0_2.Generando el data set. Elimina las filas que tengan algún campo vacío.
- Luego le aplica transformaciones a los datos:
<ul>
    <li>Transforma los valores enteros del dato 'key' a los valores strings que representan las escalas musicales</li>
    <li>Estandariza el dato string 'album_release_date' a fecha y lo guarda en el campo 'release_date'</li>
    <li>Transforma el dato 'artista_generos' (que contiene un string de géneros separados por coma):
        <ul>
            <li>Se obtienen todos los géneros sin repetir y se los estandariza a ciertos valores para reducir los datos con los que trabajamos</li>
            <li>Por cada uno de esos géneros estandar se crea una columna a modo de dummy</li>
            <li>Por cada registro, se analiza el dato 'artista_generos' y se setean a 1 los dummies correspondientes</li>               </ul>
    </li>
    <li>Dummificamos la columna 'explicit' que es de tipo booleano</li>
    <li>Eliminamos columnas innecesarias</li>
</ul>
- Definimos un subset para testeo solamente (los tracks del año 2020) y lo sacamos del dataframe principal
- Generamos dos nuevos CSV (ambos con las transformaciones): uno para el entrenamiento de los modelos y el otro para casos de testeo (los tracks del 2020)

In [120]:
from sys import maxsize #para imprimir arrays completos
import numpy as np
import pandas as pd
from datetime import datetime

# Levantamos el Dataframe

In [121]:
pathArchivoDataSet = 'tracks.csv' 
df = pd.read_csv(pathArchivoDataSet)

In [122]:
df.head()

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
0,87,0pqnGHJpmpxLKifKRmU6WP,Believer,Evolve,204346,2017-06-23,False,0.776,0.780,10,...,0.0622,0.0,0.0810,0.666,124.949,53XhwfbYqKCa1cC15pYq2q,"modern rock,rock",27642174,Imagine Dragons,4
1,86,0tgVpDi06FyKpA1z0VMD4v,Perfect,÷ (Deluxe),263400,2017-03-03,False,0.599,0.448,8,...,0.1630,0.0,0.1060,0.168,95.050,6eUKZXaKkcviH0Ku9w2n3V,"pop,uk pop",65537298,Ed Sheeran,3
2,73,2kP6fOqdSQ5CYqVH5z5844,Te Encontre,Versatilidad & Vivencias (Deluxe Edition),209960,2017-11-17,False,0.733,0.670,7,...,0.5250,0.0,0.2330,0.831,119.955,6HWumjoLO58NYInUHk9565,NaN,44866,El Vega,4
3,81,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,340706,2017-05-12,False,0.516,0.595,5,...,0.0275,0.0,0.1090,0.222,119.972,6KImCVD70vtIoJWnq6nGn3,"pop,post-teen pop",7768887,Harry Styles,4
4,85,7qiZfU4dY1lWllzX7mPBI3,Shape of You,÷ (Deluxe),233712,2017-03-03,False,0.825,0.652,1,...,0.5810,0.0,0.0931,0.931,95.977,6eUKZXaKkcviH0Ku9w2n3V,"pop,uk pop",65537298,Ed Sheeran,4


In [123]:
df.shape

(6123, 23)

# Data Cleaning

##### Analizamos los campos vacíos que tiene nuestro dataframe. Podemos ver que está completo, a excepción de la columna 'artista_generos' que tiene algunos valores faltantes

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          6123 non-null   int64  
 1   id                  6123 non-null   object 
 2   name                6123 non-null   object 
 3   album               6123 non-null   object 
 4   duration_ms         6123 non-null   int64  
 5   album_release_date  6123 non-null   object 
 6   explicit            6123 non-null   bool   
 7   danceability        6123 non-null   float64
 8   energy              6123 non-null   float64
 9   key                 6123 non-null   int64  
 10  loudness            6123 non-null   float64
 11  mode                6123 non-null   int64  
 12  speechiness         6123 non-null   float64
 13  acousticness        6123 non-null   float64
 14  instrumentalness    6123 non-null   float64
 15  liveness            6123 non-null   float64
 16  valenc

In [125]:
df_empty_genre = df[df['artista_generos'].isna()]
print('Total de registros que no tienen el dato de los géneros: ' + str(df_empty_genre['artista_name'].count()))

Total de registros que no tienen el dato de los géneros: 309


##### Tenemos 877 registros que no tienen artistas, los eliminamos del data frame principal. Ahora el dataframe nos queda sin valores nulos

In [126]:
df = df[~df['artista_generos'].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5814 entries, 0 to 6122
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          5814 non-null   int64  
 1   id                  5814 non-null   object 
 2   name                5814 non-null   object 
 3   album               5814 non-null   object 
 4   duration_ms         5814 non-null   int64  
 5   album_release_date  5814 non-null   object 
 6   explicit            5814 non-null   bool   
 7   danceability        5814 non-null   float64
 8   energy              5814 non-null   float64
 9   key                 5814 non-null   int64  
 10  loudness            5814 non-null   float64
 11  mode                5814 non-null   int64  
 12  speechiness         5814 non-null   float64
 13  acousticness        5814 non-null   float64
 14  instrumentalness    5814 non-null   float64
 15  liveness            5814 non-null   float64
 16  valenc

## Escala musical del track

##### Spotify usa números enteros para definir la escala en la que fue compuesta la canción. La relación entre ese número y las escala es la siguiente : 0 = C, 1 = C♯/D♭, 2 = D, etc (esto es 0 = Do, 1 = Do# o Re bemol, 2 = Re, 3 = Re#, etc). Realizamos una transformación para pasar del valor númerico a un string para que sea mas entendible y pueda luego dummificarse

In [127]:
#agregamos al df una nueva columna 'escala'
df['escala']= ''

In [128]:
#metodo que recibe una fila del dataset y en base al valor de la columna key le setea el valor correspondiente 
#a la columna escala
def translate_key(dataframeRow):
    #Spotify especifica que usa la siguiente notación: 0 = C, 1 = C♯/D♭, 2 = D, etc para las escalas
    #definimos un diccionario para transformar la notación numérica a una notación codificada en strings
    dicc_escalas = { 0 : 'Do', 1: 'Do#', 2: 'Re', 3: 'Re#', 4: 'Mi', 5: 'Fa', 6: 'Fa#', 7: 'Sol', 8: 'Sol#', 9: 'La', 10: 'La#', 11: 'Si', 12: 'Si#', 13: 'Do' }
    dataframeRow.escala = dicc_escalas[dataframeRow.key]
    return dataframeRow

In [129]:
#aplicamos la traducción del valor numérico que representa la escala a un string
df = df.apply(translate_key, axis=1)

In [130]:
df.loc[:, ['key','escala']].head()

,key,escala
0,10,La#
1,8,Sol#
3,5,Fa
4,1,Do#
5,9,La


In [131]:
#sacamos del dataframe la columna original 'key'
df = df.drop(labels='key', axis = 1)

## Año de creación del track

##### Generamos una columna nueva para almacenar la fecha de lanzamiento del track en formato DateTime

In [132]:
df['release_date'] = datetime.today()

In [133]:
#metodo que recibe una fila del data frame y transforma la fecha en formato string de la columna album_release_date
#al formato DateTime y lo guarda en release_date
def cast_string_to_date(dataframeRow):
    try:
        dataframeRow.release_date = datetime.strptime(dataframeRow.album_release_date, '%Y-%m-%d').date()
    except:
        try:
            dataframeRow.release_date = datetime.strptime(dataframeRow.album_release_date, '%Y').date()
        except:
            print("No se pudo convertir string en fecha: " + dataframeRow.album_release_date)
        
    return dataframeRow

In [134]:
df = df.apply(cast_string_to_date, axis=1)

##### Eliminamos la columna fecha en formato string

In [135]:
df = df.drop(labels='album_release_date', axis = 1)

## Géneros

#### Spotify asocia los géneros a los artistas, no a sus tracks. Es por eso que en el dataframe, todas las filas que tengan el mismo artista tendrán los mismos géneros


In [136]:
artist_name = 'Divididos'
artist_divididos = df[df['artista_name']==artist_name]
print('Tenemos ' + str(artist_divididos.shape[0]) + ' registros (tracks) para el artista ' + artist_name)

Tenemos 8 registros (tracks) para el artista Divididos


In [137]:
#Podemos ver que el género está dado por artista, obteniendo el dato artista_generos y viendo que solo tiene un valor único
#para todos los registros (cada registro es un track)
artist_divididos['artista_generos'].unique()

array(['argentine rock,latin alternative,latin rock,rock en espanol,rock nacional,ska argentino'],
      dtype=object)

In [138]:
#Obteniendo los géneros sin repetir, obtenemos 3413
array_of_strings = df['artista_generos'].unique()
print('Buscando los conjuntos de géneros sin repetir tenemos en el data frame, un total de: ' + str(array_of_strings.shape[0]))

Buscando los conjuntos de géneros sin repetir tenemos en el data frame, un total de: 959


In [139]:
#Ejemplo de un dato género
array_of_strings[0]

'modern rock,rock'

##### Vamos a tener que separar el conjunto de géneros en géneros individuales (sin repetir)

In [140]:
#generamos un Set para obtener los géneros individuales
genres_set = set()

for concatenated_genres in array_of_strings:
    split_genres = concatenated_genres.split(',')
    
    for genre in split_genres:
        genres_set.add(genre)

print('Vemos que en total son ' + str(len(genres_set)) + ' géneros')

Vemos que en total son 608 géneros


In [141]:
genres_set

{'acoustic chill',
 'acoustic pop',
 'adult standards',
 'afro dancehall',
 'afro house',
 'afrofuturism',
 'afrofuturismo brasileiro',
 'afroswing',
 'album rock',
 'alternative dance',
 'alternative hip hop',
 'alternative metal',
 'alternative pop rock',
 'alternative r&b',
 'alternative rock',
 'ambient folk',
 'ambient pop',
 'ambient worship',
 'andean',
 'anime',
 'anime latino',
 'anime rock',
 'anime score',
 'anthem worship',
 'antiviral pop',
 'argentine alternative rock',
 'argentine hip hop',
 'argentine indie',
 'argentine indie rock',
 'argentine metal',
 'argentine punk',
 'argentine reggae',
 'argentine rock',
 'argentine telepop',
 'art pop',
 'art rock',
 'asian american hip hop',
 'asmr',
 'atl hip hop',
 'atl trap',
 'atmosphere',
 'aussietronica',
 'austindie',
 'australian alternative pop',
 'australian dance',
 'australian electropop',
 'australian hip hop',
 'australian house',
 'australian pop',
 'australian reggae fusion',
 'azonto',
 'bachata',
 'background 

##### Hacemos una reducción de los géneros, haciendo una transformación de los mismos a unas categorías definidas

In [142]:
#metodo que recibe un string con un género musical y lo reduce a un género musical mas general
def reduce_genre(general_genre):
    genre = ''
    if(genre == '' and 'rock' in general_genre): return 'rock'
    if(genre == '' and 'grunge' in general_genre): return 'rock'
    if(genre == '' and 'pop' in general_genre): return 'pop'
    if(genre == '' and 'sing' in general_genre): return 'pop'
    if(genre == '' and 'urban' in general_genre): return 'pop'
    if(genre == '' and 'boy' in general_genre): return 'pop'
    if(genre == '' and 'girl' in general_genre): return 'pop'
    if(genre == '' and 'disney' in general_genre): return 'pop'
    if(genre == '' and 'alternative' in general_genre): return 'alternative'
    if(genre == '' and 'down' in general_genre): return 'alternative'
    if(genre == '' and 'drill' in general_genre): return 'alternative'
    if(genre == '' and 'emo' in general_genre): return 'alternative'
    if(genre == '' and 'room' in general_genre): return 'alternative'
    if(genre == '' and 'show' in general_genre): return 'alternative'
    if(genre == '' and 'hip hop' in general_genre): return 'hip_hop'
    if(genre == '' and 'hiphop' in general_genre): return 'hip_hop'
    if(genre == '' and 'hip-hop' in general_genre): return 'hip_hop'
    if(genre == '' and 'ska' in general_genre): return 'ska'
    if(genre == '' and 'indie' in general_genre): return 'indie'
    if(genre == '' and 'jazz' in general_genre): return 'jazz'
    if(genre == '' and 'swing' in general_genre): return 'jazz'
    if(genre == '' and 'groove' in general_genre): return 'jazz'
    if(genre == '' and 'metal' in general_genre): return 'metal'
    if(genre == '' and 'industrial' in general_genre): return 'metal'
    if(genre == '' and 'trap' in general_genre): return 'trap'
    if(genre == '' and 'ska' in general_genre): return 'ska'
    if(genre == '' and 'reggaeton' in general_genre): return 'reggaeton'
    if(genre == '' and 'perreo' in general_genre): return 'reggaeton'
    if(genre == '' and 'bachata' in general_genre): return 'bachata'
    if(genre == '' and 'bolero' in general_genre): return 'bachata'
    if(genre == '' and 'reggae' in general_genre): return 'reggae'
    if(genre == '' and 'blues' in general_genre): return 'blues'
    if(genre == '' and 'rap' in general_genre): return 'rap'
    if(genre == '' and 'disco' in general_genre): return 'disco'
    if(genre == '' and 'folk' in general_genre): return 'folk'
    if(genre == '' and 'chamame' in general_genre): return 'folk'
    if(genre == '' and 'folc' in general_genre): return 'folk'
    if(genre == '' and 'andean' in general_genre): return 'folk'
    if(genre == '' and 'punk' in general_genre): return 'punk'
    if(genre == '' and 'soul' in general_genre): return 'soul'
    if(genre == '' and 'r&b' in general_genre): return 'r&b'
    if(genre == '' and 'techno' in general_genre): return 'techno'
    if(genre == '' and 'trance' in general_genre): return 'trance'
    if(genre == '' and 'rumba' in general_genre): return 'rumba'
    if(genre == '' and 'gothic' in general_genre): return 'gothic'
    if(genre == '' and 'cumbia' in general_genre): return 'cumbia'
    if(genre == '' and 'cuarteto' in general_genre): return 'cuarteto'
    if(genre == '' and 'movie' in general_genre): return 'movie'
    if(genre == '' and 'electr' in general_genre): return 'electronic'
    if(genre == '' and 'tronica' in general_genre): return 'electronic'
    if(genre == '' and 'tronic' in general_genre): return 'electronic'
    if(genre == '' and 'house' in general_genre): return 'house'
    if(genre == '' and 'deep' in general_genre): return 'house'
    if(genre == '' and 'christian' in general_genre): return 'christian'
    if(genre == '' and 'worsh' in general_genre): return 'christian'    
    if(genre == '' and 'cristi' in general_genre): return 'christian'    
    if(genre == '' and 'under' in general_genre): return 'under'
    if(genre == '' and 'groove' in general_genre): return 'groove'
    if(genre == '' and 'funk' in general_genre): return 'funk'
    if(genre == '' and 'anime' in general_genre): return 'anime'
    if(genre == '' and 'ambient' in general_genre): return 'ambient'
    if(genre == '' and 'atmosphere' in general_genre): return 'ambient'
    if(genre == '' and 'fondo' in general_genre): return 'ambient'
    if(genre == '' and 'chill' in general_genre): return 'ambient'
    if(genre == '' and 'quiet' in general_genre): return 'ambient'
    if(genre == '' and 'slee' in general_genre): return 'ambient'
    if(genre == '' and 'sound' in general_genre): return 'ambient'
    if(genre == '' and 'noise' in general_genre): return 'ambient'    
    if(genre == '' and 'country' in general_genre): return 'country'
    if(genre == '' and 'dance' in general_genre): return 'dance'
    if(genre == '' and 'bolero' in general_genre): return 'bolero'
    if(genre == '' and 'bossa nova' in general_genre): return 'bossa_nova'
    if(genre == '' and 'instrumental' in general_genre): return 'instrumental'
    if(genre == '' and 'melod' in general_genre): return 'instrumental'    
    if(genre == '' and 'chans' in general_genre): return 'instrumental'    
    if(genre == '' and 'cassical' in general_genre): return 'classical'
    if(genre == '' and 'pianissimo' in general_genre): return 'classical'
    if(genre == '' and 'piano' in general_genre): return 'classical'    
    if(genre == '' and 'hardcore' in general_genre): return 'hardcore'
    if(genre == '' and 'deep' in general_genre): return 'deep'
    if(genre == '' and 'dubstep' in general_genre): return 'dubstep'
    if(genre == '' and 'step' in general_genre): return 'dubstep'
    if(genre == '' and 'dub' in general_genre): return 'dub'
    if(genre == '' and 'experimental' in general_genre): return 'experimental'
    if(genre == '' and 'flamenco' in general_genre): return 'flamenco'
    if(genre == '' and 'freestyle' in general_genre): return 'freestyle'
    if(genre == '' and 'mambo' in general_genre): return 'mambo'
    if(genre == '' and 'merengue' in general_genre): return 'merengue'
    if(genre == '' and 'salsa' in general_genre): return 'salsa'
    if(genre == '' and 'tango' in general_genre): return 'tango'
    if(genre == '' and 'soul' in general_genre): return 'soul'
    if(genre == '' and 'new' in general_genre): return 'neo'
    if(genre == '' and 'neo' in general_genre): return 'neo'
    if(genre == '' and 'nu ' in general_genre): return 'neo'
    if(genre == '' and 'post' in general_genre): return 'neo'
    if(genre == '' and 'futur' in general_genre): return 'neo'
    if(genre == '' and 'opera' in general_genre): return 'opera'
    if(genre == '' and 'broadway' in general_genre): return 'opera'
    if(genre == '' and 'rave' in general_genre): return 'rave'
    if(genre == '' and 'progres' in general_genre): return 'progressive'
    if(genre == '' and 'salsa' in general_genre): return 'salsa'
    if(genre == '' and 'samba' in general_genre): return 'samba'
    if(genre == '' and 'swing' in general_genre): return 'swing'
    if(genre == '' and 'trance' in general_genre): return 'trance'
    if(genre == '' and 'world' in general_genre): return 'world'
    if(genre == '' and 'dembow' in general_genre): return 'world'
    if(genre == '' and 'internat' in general_genre): return 'world'
    if(genre == '' and 'k-pop' in general_genre): return 'k_pop'          
    
    return genre            

In [143]:
reduce_genre('argentine telepop,folklore argentino,latin,latin pop')

'pop'

In [144]:
#generamos un nuevo set, aplicándole al set que contiene los géneros generales la transformación para obtener nuestros géneros
#seleccionados como categorias
set_filtrado = set()

for x in genres_set:
    genero_normalizado = reduce_genre(x)
    set_filtrado.add(genero_normalizado)    

##### Con el set de géneros obtenidos los agregamos al data frame como nuevas columnas dummies

In [145]:
df['genre_cuarteto'] = 0
df['genre_bachata'] = 0
df['genre_k_pop'] = 0
df['genre_soul'] = 0
df['genre_rock'] = 0
df['genre_pop'] = 0
df['genre_alternative'] = 0
df['genre_hip_hop'] = 0
df['genre_ska'] = 0
df['genre_indie'] = 0
df['genre_jazz'] = 0
df['genre_metal'] = 0
df['genre_trap'] = 0
df['genre_reggaeton'] = 0
df['genre_reggae'] = 0
df['genre_blues'] = 0
df['genre_rap'] = 0
df['genre_disco'] = 0
df['genre_folk'] = 0
df['genre_punk'] = 0
df['genre_soul'] = 0
df['genre_r&b'] = 0
df['genre_techno'] = 0
df['genre_trance'] = 0
df['genre_rumba'] = 0
df['genre_gothic'] = 0
df['genre_cumbia'] = 0
df['genre_movie'] = 0
df['genre_electronic'] = 0
df['genre_house'] = 0
df['genre_christian'] = 0
df['genre_under'] = 0
df['genre_groove'] = 0
df['genre_funk'] = 0
df['genre_anime'] = 0
df['genre_ambient'] = 0
df['genre_country'] = 0
df['genre_dance'] = 0
df['genre_bolero'] = 0
df['genre_bossa_nova'] = 0
df['genre_instrumental'] = 0
df['genre_classical'] = 0
df['genre_melodic'] = 0
df['genre_hardcore'] = 0
df['genre_deep'] = 0
df['genre_dubstep'] = 0
df['genre_dub'] = 0
df['genre_experimental'] = 0
df['genre_flamenco'] = 0
df['genre_freestyle'] = 0
df['genre_mambo'] = 0
df['genre_merengue'] = 0
df['genre_salsa'] = 0
df['genre_tango'] = 0
df['genre_neo'] = 0
df['genre_opera'] = 0
df['genre_rave'] = 0
df['genre_progressive'] = 0
df['genre_samba'] = 0
df['genre_swing'] = 0
df['genre_trance'] = 0
df['genre_world'] = 0    
df['genre_others'] = 0    

In [146]:
#metodo que recibe una fila del data frame, analiza los generos asociados al artista y setea a 1 los géneros categóricos 
#dummificados
def set_dummy_values(dataframeRow):
    #spliteamos el listado de generos, separados por coma
    row_genres_split = dataframeRow.artista_generos.split(',')
    row_genres_set = set()
    has_genre = False
    
    #por cada genero aplicamos la normalizacion de géneros para reducirlos a las categorías que definimos
    for x in row_genres_split:
        genre = reduce_genre(x)
        if genre != '' and not has_genre:
            row_genres_set.add(genre)
            has_genre= True
    
    #obtenidas las categorías, seteamos a 1 los valores en las columnas dummies
    if(has_genre):
        for x in row_genres_set:
            dataframeRow['genre_'+x] = 1
    else:
        dataframeRow['genre_others'] = 1
    return dataframeRow

In [147]:
#aplicamos el seteo de los dummies de los géneros
df = df.apply(set_dummy_values, axis=1)

##### Eliminamos la columna que trae todos los géneros concatenados

In [148]:
#df = df.drop(labels='artista_generos', axis = 1)

# Dummificamos la columna Explicit

##### Esta columna es booleana, la pasamos a numérica para dummificarla

In [149]:
df['explicit'] = df['explicit'].astype(int)

# Eliminamos otras columnas que no son de utilidad

In [150]:
df = df.drop(labels='id', axis = 1)
df = df.drop(labels='artista_id', axis = 1)

# Definimos un subset de datos para testeo

### Separamos del set de datos todos los temas que fueron lanzados en el año 2020, para tener un subset de datos exclusivos para el testeo

In [151]:
fecha_inicio_testing =  datetime.strptime('2020-01-01', '%Y-%m-%d').date()
fecha_final_testing =  datetime.strptime('2020-12-31', '%Y-%m-%d').date()

In [152]:
df_test_exclusivo = df[(df['release_date'] >= fecha_inicio_testing) & (df['release_date'] <= fecha_final_testing)]

In [153]:
df_test_exclusivo.shape

(117, 82)

##### Quitamos del dataframe de entrenamiento el subset exclusivo de testing

In [154]:
df = df.drop(axis=0, index=df_test_exclusivo.index)

In [155]:
df.shape

(5697, 82)

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5697 entries, 0 to 5999
Data columns (total 82 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          5697 non-null   int64  
 1   name                5697 non-null   object 
 2   album               5697 non-null   object 
 3   duration_ms         5697 non-null   int64  
 4   explicit            5697 non-null   int32  
 5   danceability        5697 non-null   float64
 6   energy              5697 non-null   float64
 7   loudness            5697 non-null   float64
 8   mode                5697 non-null   int64  
 9   speechiness         5697 non-null   float64
 10  acousticness        5697 non-null   float64
 11  instrumentalness    5697 non-null   float64
 12  liveness            5697 non-null   float64
 13  valence             5697 non-null   float64
 14  tempo               5697 non-null   float64
 15  artista_generos     5697 non-null   object 
 16  artist

# Generamos el CSV con los datos limpios

In [157]:
tracks_csv_name = 'tracks_clean.csv'
df.to_csv(tracks_csv_name,index=False,encoding='utf-8')

# Generamos un CSV con los datos de testeo

In [158]:
tracks_csv_name = 'tracks_clean_test.csv'
df_test_exclusivo.to_csv(tracks_csv_name,index=False,encoding='utf-8')